In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import os

In [4]:
data_path = r'C:\Users\Usuario\Desktop\Projects\Maven Sales Challenge\data'

def load(data_folder, filename):
    df = pd.read_csv(os.path.join(data_folder, filename))
    return df

accounts = load(data_path, 'accounts.csv')
products = load(data_path, 'products.csv')
sales_pipeline = load(data_path, 'sales_pipeline.csv')
sales_teams = load(data_path, 'sales_teams.csv')

tables = [accounts, products, sales_pipeline, sales_teams] 

counter = 1
for table in tables:
    print(f"details of table {counter}")
    print(table.describe(), '\n')
    print(table.info(), '\n')
    print('*' * 100)


details of table 1
       year_established       revenue     employees
count         85.000000     85.000000     85.000000
mean        1996.105882   1994.632941   4660.823529
std            8.865427   2169.491436   5715.601198
min         1979.000000      4.540000      9.000000
25%         1989.000000    497.110000   1179.000000
50%         1996.000000   1223.720000   2769.000000
75%         2002.000000   2741.370000   5595.000000
max         2017.000000  11698.030000  34288.000000 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-nul

### 1. Are older companies (pre-1996) generating higher sales revenue than newer ones?


In [45]:
df = pd.merge(accounts, sales_pipeline, on='account', how='left')
df['company_age_group'] = np.where(df['year_established'] > 1996, 'Newer (>=1996)', 'Older (<1996)')

grouped = df.groupby('company_age_group').agg({'close_value' : 'sum'}).reset_index()

fig = px.bar(
    grouped,
    x='company_age_group',
    y='close_value',
    title='Total Close Value by Company Age Group',
    labels={'company_age_group':'Company Age Group', 'close_value': 'Close Value'},
    text='close_value',
    template='plotly_dark',
    color='company_age_group',  # Needed if using multiple colors
    color_discrete_sequence=['crimson', 'lightskyblue']  # Custom colors
)
fig.update_layout(
    width=800,
    height=500
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

### 2. Do larger companies (by employee count) tend to close bigger deals?

In [ ]:


# Merged and grouped DataFrame (already created)
df = pd.merge(accounts, sales_pipeline, on='account', how='left')

grouped = df.groupby(['account', 'employees']).agg({
    'close_value': 'mean'
}).reset_index()

grouped = grouped.sort_values(['employees'], ascending=False).head(10)

# Create the figure
fig = go.Figure()

# Add bar trace for employees
fig.add_trace(go.Bar(
    x=grouped['account'],
    y=grouped['employees'],
    name='Number of Employees',
    marker_color='lightskyblue',
    yaxis='y1'
))

# Add line trace for average close_value
fig.add_trace(go.Scatter(
    x=grouped['account'],
    y=grouped['close_value'],
    name='Avg Close Value',
    mode='lines+markers',
    marker=dict(color='crimson'),
    line=dict(shape='spline', color='crimson'),
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title='Employee Count (Bar) vs. Average Close Value (Line) per Account',
    xaxis=dict(title='Company'),
    yaxis=dict(title='Number of Employees', side='left'),
    yaxis2=dict(title='Avg Close Value', overlaying='y', side='right'),
    template='plotly_dark',
    width=1400,
    height=500,
    legend=dict(x=1, y=1, xanchor='right', yanchor='top'),
)

fig.show()


### 3. which sales_agent has the highest total close_value for accounts with above-average revenue (>$1994.63 million), and how does this vary by regional_office?

In [49]:
# Step 1: Filter accounts with above-average revenue
high_revenue_accounts = accounts[accounts['revenue'] > 1994.63]

# Step 2: Merge with sales pipeline to get close_value
merged = pd.merge(high_revenue_accounts, sales_pipeline, on='account', how='inner')

# Step 3: Merge with sales team data to get regional office info
full_df = pd.merge(merged, sales_teams, on='sales_agent', how='left')

# Step 4: Group and aggregate
grouped = full_df.groupby(['sales_agent', 'regional_office'])['close_value'].sum().reset_index()

# Step 5: Sort by close_value to find the top agent(s)
top_agents = grouped.sort_values('close_value', ascending=False)


fig = px.bar(top_agents,
             x='sales_agent',
             y='close_value',
             color='regional_office',
             title='Total Close Value by Sales Agent (High-Revenue Accounts)',
             labels={'close_value': 'Total Close Value'},
             template='plotly_dark',
             text='close_value')

fig.update_layout(width=1400, height=500)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()


### 4. Are larger companies (by employee count) also the highest earners in revenue? Is there a correlation between company size and revenue?

In [50]:
px.scatter(accounts,
           x='employees',
           y='revenue',
           size='revenue',
           hover_name='account',
           title='Revenue vs. Number of Employees (Bubble Size = Revenue)',
           template='plotly_dark',
           color='sector')


### 5. Which products generate the most revenue? Should sales efforts be concentrated on certain high-performing products?

In [56]:
grouped = sales_pipeline.groupby('product')['close_value'].sum().reset_index()
grouped = grouped.sort_values('close_value', ascending=False)

px.bar(grouped,
       x='product',
       y='close_value',
       title='Total Sales by Product',
       template='plotly_dark',
       text='close_value').update_traces(textposition='outside')


### 6. Which sales agents perform best in each regional office? Are there underperforming regions or agents that need support?

In [59]:
df = pd.merge(sales_pipeline, sales_teams, on='sales_agent', how='left')
grouped = df.groupby(['sales_agent', 'regional_office'])['close_value'].sum().reset_index()

px.density_heatmap(grouped,
                   x='sales_agent',
                   y='regional_office',
                   z='close_value',
                   color_continuous_scale='Viridis',
                   title='Sales Agent Performance by Region',
                   template='plotly_dark')
